In [1]:
pip install music21

In [2]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils

**Read a midi File**

In [15]:
midi = converter.parse('../input/mid-files/eyesonme.mid')

In [16]:
midi

In [17]:
# midi.show('midi')

In [8]:
# midi.show('text')

In [18]:
elements_to_parse = midi.flat.notes

In [19]:
len(elements_to_parse)

In [12]:
# for e in elements_to_parse:
#     print(e, e.offset)

In [21]:
notes_demo = []

for ele in elements_to_parse:
    # If element is a Note, then store it's pitch
    if isinstance(ele, note.Note):
        notes_demo.append(str(ele.pitch))
        
    # If element is a chord, split each note of chord and join with +
    elif isinstance(ele, chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder))

    

In [22]:
len(notes_demo)

**Preprocessing all Files**

In [23]:
notes = [] 

for file in glob.glob('../input/mid-files/*.mid'):
    midi = converter.parse(file)
    
    print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    for ele in elements_to_parse:
        # If element is a Note, then store it's pitch
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))

        # If element is a chord, split each note of chord and join with +
        elif isinstance(ele, chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))


In [24]:
len(notes)

In [25]:
with open("notes", 'wb') as filepath:
    pickle.dump(notes, filepath)

In [26]:
with open("notes", 'rb') as f:
    notes = pickle.load(f)

In [27]:
n_vocab = len(set(notes))

In [28]:
print("Total notes - ", len(notes))
print("Unique notes - ", n_vocab)

In [29]:
print(notes[100:300])

**Prepare Sequential Data for LSTM**

In [30]:
# How many elements LSTM input should consider
sequence_length = 100

In [31]:
pitchnames = sorted(set(notes))

In [32]:
# Mapping between ele to int value
ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames))

In [33]:
network_input = []
network_output = []

In [34]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i + sequence_length] # contains sequence lenth number values
    seq_out = notes[i + sequence_length]
    
    network_input.append([ ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [35]:
# No. of examples
n_patterns = len(network_input)
print(n_patterns)

In [36]:
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

In [37]:
normalised_network_input = network_input/(float)(n_vocab)

In [38]:
# Network output are the classes, encode into one hot vector
network_output = np_utils.to_categorical(network_output)

In [39]:
print(normalised_network_input.shape)
print(network_output.shape)

**Create Model**

In [47]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [42]:
model = Sequential()
model.add( LSTM(units=512, 
                input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2]),
               return_sequences = True) )

model.add( Dropout(0.3) )
model.add( LSTM(units=512, return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(units=512) )
model.add( Dense(256))
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation = "softmax") )   

In [43]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam")

In [44]:
model.summary()

In [45]:
checkpoint = ModelCheckpoint("model.hdf5", monitor = 'loss', verbose = 0, save_best_only = True, mode = 'min')


model_his = model.fit(normalised_network_input, network_output, epochs = 100, batch_size = 64, callbacks = [checkpoint])

In [48]:
model = load_model("../input/mid-modeltrained/model.hdf5")

**Predictions**

In [52]:
sequence_length = 100
network_input = []


for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i + sequence_length] # contains sequence lenth number values
    network_input.append([ ele_to_int[ch] for ch in seq_in])

In [63]:
start = np.random.randint(len(network_input) - 1)

In [73]:
# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

# Initial Pattern
pattern = network_input[start]
prediction_output = []

# Generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    
    prediction = model.predict(prediction_input, verbose = 0)
    
    idx = np.argmax(prediction)
    
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]
    

In [74]:
print(prediction_output)

**Create Midi File**

In [78]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # If the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) # create Note object for each node in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        new_chord = chord.Chord(temp_notes) # Create the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    else:
        # If the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.
        output_notes.append(new_note)
    
    offset += 0.5 

In [80]:
# Create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid")